Verificación de calidad de datos con Great Expectations
=======================================================

Great Expectations
------------------

`Great expentations` es una libreria en `Python` y una utilidad de linea de comandos que provee un lenguaje declarativo para y flexible para describir nuestras expectativas sobre como los datos deben lucir. Permite validar y documentar la calidad de los datos para luego facilmente comunicar los resultados.

Podemos instalar `Great Expectations` en `Python` como sigue:

```
pip install great_expectations
```

La librería Great Expectations’ más de 50 `expectations` ya listas para utilizar:

- `expect_column_values_to_not_be_null`
- `expect_column_values_to_match_regex`
- `expect_column_values_to_be_unique`
- `expect_column_values_to_match_strftime_format`
- `expect_table_row_count_to_be_between`
- `expect_column_median_to_be_between`

Para más información sobre esta librería puede ver [su sitio web](https://docs.greatexpectations.io/docs/).

### Preparación del notebook

Instalamos las librerías necesarias:

In [ ]:
!wget https://raw.githubusercontent.com/santiagxf/E72102/master/docs/develop/prep/code/expectations.txt \
    --quiet --no-clobber
!pip install -r expectations.txt --quiet

In [22]:
import warnings
import ruamel

warnings.filterwarnings("ignore")
warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

Ejemplo
-------
Utilizaremos el conjunto de datos [NYC Taxi data](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page). Cada registro en los datos corresponde a un viaje en taxi y contiene información como el lugar donde se toma el taxi y el lugar donde el pasajero se baja del mismo, el monto del pago y el número de pasajeros, entre otros.

Si bien este conjunto de datos es actualizado cada mes, en nuestro caso solu utilizaremos dos archivos CSV correspondiente a 2 meses, cada uno con una muestra de 10,000 filas del conjunto Yellow Taxi Trip Records:

- yellow_tripdata_sample_2019-01.csv: una muestra de los datos de taxis de enero de 2019

- yellow_tripdata_sample_2019-02.csv: una muestra de los datos de taxis de febrero de 2019

In [26]:
import pandas as pd
import glob

df = pd.concat(map(lambda file: pd.read_csv(file), glob.glob("datasets/nyc_taxi/*.csv")))
df.head(5)

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-15 03:36:12,2019-01-15 03:42:19,1,1.0,1,N,230,48,1,6.5,0.5,0.5,1.95,0.0,0.3,9.75,NaN
1,1,2019-01-25 18:20:32,2019-01-25 18:26:55,1,0.8,1,N,112,112,1,6.0,1.0,0.5,1.55,0.0,0.3,9.35,0.0
2,1,2019-01-05 06:47:31,2019-01-05 06:52:19,1,1.1,1,N,107,4,2,6.0,0.0,0.5,0.00,0.0,0.3,6.80,NaN
3,1,2019-01-09 15:08:02,2019-01-09 15:20:17,1,2.5,1,N,143,158,1,11.0,0.0,0.5,3.00,0.0,0.3,14.80,NaN
4,1,2019-01-25 18:49:51,2019-01-25 18:56:44,1,0.8,1,N,246,90,1,6.5,1.0,0.5,1.65,0.0,0.3,9.95,0.0


### Configurando Great Expectations

En Great Expectations es necesario crear un contexto de datos para comenzar a trabajar. El contexto administra la configuración del proyecto en el que estamos trabajando. Para inicializar nuestro proyecto, utilizaremos el siguiente comando:

In [ ]:
printf "Y\nn\n" | great_expectations init --no-view

Una vez que la librería está configurada correctamente, podemos generar uno de los objetos más utilizados en esta librería que es un `DataContext`. Este objeto nos permite interactuar con todas las funcionalidades disponibles:

In [1]:
import great_expectations as ge

context = ge.get_context()

### Diseñando expectativas

Para diseñar nuestras expectativas, una de las formas más sencillas es realizarlo de forma interactiva, donde podemos ir validando las mismas a medida que las diseñamos:

In [27]:
gedf = ge.from_pandas(df)

In [28]:
gedf.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-15 03:36:12,2019-01-15 03:42:19,1,1.0,1,N,230,48,1,6.5,0.5,0.5,1.95,0.0,0.3,9.75,NaN
1,1,2019-01-25 18:20:32,2019-01-25 18:26:55,1,0.8,1,N,112,112,1,6.0,1.0,0.5,1.55,0.0,0.3,9.35,0.0
2,1,2019-01-05 06:47:31,2019-01-05 06:52:19,1,1.1,1,N,107,4,2,6.0,0.0,0.5,0.00,0.0,0.3,6.80,NaN
3,1,2019-01-09 15:08:02,2019-01-09 15:20:17,1,2.5,1,N,143,158,1,11.0,0.0,0.5,3.00,0.0,0.3,14.80,NaN
4,1,2019-01-25 18:49:51,2019-01-25 18:56:44,1,0.8,1,N,246,90,1,6.5,1.0,0.5,1.65,0.0,0.3,9.95,0.0


In [63]:
gedf["payment_type"].unique()

array([1, 2, 4, 3])

Cuando ejecutamos una expectativa, se validará inmediatamente con el conjunto de datos. El objeto que devuelve este método contendrá el resultado y una lista de valores que no cumplen con la expectativa. Esta información nos ayuda a concentrarnos en datos que no esperamos tener muy rápidamente, eliminando muchas conjeturas de la exploración de datos.

Primero demosle un nombre al conjunto de expectativas que crearemos para este set de datos:

In [58]:
gedf.expectation_suite_name = 'yellow_tripdata_sample_quality'

La siguiente expectativa indica que la cantidad de pasajeros del viaje debe ser al menos 1. Como vemos, alrededor del 7.8% de los datos no cumplen con esta expectativa y el valor `success` es `false`:

In [59]:
gedf.expect_column_values_to_be_between("passenger_count", min_value=1)

{
  "result": {
    "element_count": 20000,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 1579,
    "unexpected_percent": 7.8950000000000005,
    "unexpected_percent_total": 7.8950000000000005,
    "unexpected_percent_nonmissing": 7.8950000000000005,
    "partial_unexpected_list": [
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0
    ]
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": false,
  "meta": {}
}

Es común encontrar problemas de datos donde no es necesario garantizar el 100% de adherencia. En estos casos, considere usar un parámetro `mostly`. Este parámetro es una opción para todas las Expectativas y permite controlar el nivel de margen tolerancia en el que la expectativa podría no cumplirse. Mire como en el próximo ejemplo, el parametro `mostly` hace que la expecativa retorne `true` en la propiedad `success`:

In [60]:
gedf.expect_column_values_to_be_between("passenger_count", min_value=1, mostly=.90)

{
  "result": {
    "element_count": 20000,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 1579,
    "unexpected_percent": 7.8950000000000005,
    "unexpected_percent_total": 7.8950000000000005,
    "unexpected_percent_nonmissing": 7.8950000000000005,
    "partial_unexpected_list": [
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0
    ]
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "meta": {}
}

Podemos también agregar documentación a nuestras expectativas para futura referencia:

In [61]:
gedf.expect_column_values_to_be_between("passenger_count", min_value=1,     
                                        meta = {
                                            "notes": {
                                               "content": [ "Todos los viajes deben tener al menos un pasajero." ],
                                               "format": "markdown",
                                               "source": "expectations"
                                            }
                                        })

{
  "result": {
    "element_count": 20000,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 1579,
    "unexpected_percent": 7.8950000000000005,
    "unexpected_percent_total": 7.8950000000000005,
    "unexpected_percent_nonmissing": 7.8950000000000005,
    "partial_unexpected_list": [
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0
    ]
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": false,
  "meta": {
    "notes": {
      "content": [
        "Todos los viajes deben tener al menos un pasajero."
      ],
      "format": "markdown",
      "source": "expectations"
    }
  }
}

Creemos algunas otras expectativas:

In [72]:
gedf.expect_column_values_to_be_in_set("store_and_fwd_flag", ['Y', 'N'],     
                                        meta = {
                                            "notes": {
                                               "content": [ "Acepta valores de Si y No." ],
                                               "format": "markdown",
                                               "source": "expectations"
                                            }
                                        })
gedf.expect_column_values_to_be_in_set("payment_type", [1, 2, 3, 4],     
                                        meta = {
                                            "notes": {
                                               "content": [ "Solo formas de pago 1, 2, 3, y 5 son validas" ],
                                               "format": "markdown",
                                               "source": "expectations"
                                            }
                                        })
gedf.expect_column_pair_values_A_to_be_greater_than_B(column_A="dropoff_datetime", 
                                                      column_B="pickup_datetime", 
                                                      or_equal=True,
                                                      meta = {
                                                         "notes": {
                                                            "content": [ "Las fechas de inicio y fin de viaje deben coincidir." ],
                                                            "format": "markdown",
                                                            "source": "expectations"
                                                         }
                                                      })

print("Listo")

Listo


Una vez que completamos nuestras expectativas, podemos extraer toda la configuración como sigue:

In [73]:
expectations = gedf.get_expectation_suite(discard_failed_expectations=False)

Podemos persistir estas expectativas como un archivo `JSON` que luego podemos cargar y aplicar a nuevos lotes de datos cada vez que se realiza el proceso de preparación de datos:

In [74]:
import json

with open( "great_expectations/expectations/nyc_data/data_quality.json", "w") as f:
    f.write(
        json.dumps(expectations.to_json_dict())
    )

### Configurando datos como un origen de datos

Si bien leer los datos desde un `DataFrame` de `Pandas` es práctico, si registramos este conjunto de datos como un `origen de datos` dentro de Great Expectations, podremos acceder a otras funcionalidades. 

Identificamos el directorio donde se encuentran los datos:

In [2]:
datasource_name = "nyc_taxi"
base_data_directory = "../datasets/nyc_taxi"

Configuraremos un nuevo origen de datos donde `great_expectations` podrá acceder a los datos:

In [13]:
example_yaml = f"""
name: {datasource_name}
class_name: Datasource
execution_engine:
  class_name: PandasExecutionEngine
data_connectors:
  nyc_taxi_data_connector:
    class_name: InferredAssetFilesystemDataConnector
    base_directory: {base_data_directory}
    default_regex:
      group_names: 
        - data_asset_name
        - year
        - month
      pattern: (.*)_(\\d{{4}})-(\\d{{2}})\\.csv
"""
print(example_yaml)


name: nyc_taxi
class_name: Datasource
execution_engine:
  class_name: PandasExecutionEngine
data_connectors:
  nyc_taxi_data_connector:
    class_name: InferredAssetFilesystemDataConnector
    base_directory: ../nyc_taxi
    default_regex:
      group_names: 
        - data_asset_name
        - year
        - month
      pattern: (.*)_(\d{4})-(\d{2})\.csv



In [130]:
context.test_yaml_config(yaml_config=example_yaml)

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: PandasExecutionEngine
Data Connectors:
	nyc_taxi_data_connector : InferredAssetFilesystemDataConnector

	Available data_asset_names (1 of 1):
		yellow_tripdata_sample (2 of 2): ['yellow_tripdata_sample_2019-01.csv', 'yellow_tripdata_sample_2019-02.csv']

	Unmatched data_references (0 of 0):[]



Agregamos el origen de datos:

In [131]:
from ruamel import yaml

ds = context.add_datasource(**yaml.load(example_yaml))

### Creando expectativas utilizando un Profiler

Un `profile` o perfil de datos es una forma de caracterizar un conjunto de datos de forma automática. Permite, utilizando una pequeña porción de datos, extraer las características que existen en este conjunto de datos para escribir las expectativas. Esto significa que esta pequeña porción de datos debe estar curada y contener las cualidades que queremos tener.

> Nota: El hecho de utilizar una pequeña porción de datos en lugar de todo el set de datos completo radica en que en ambientes productivos, los conjuntos de datos podrían ser muy grandes. Es común utilizar una pequeña porción de los datos para inferir las propiedades y luego si validarlas sobre todo el conjunto de datos.

In [117]:
from great_expectations.profile.user_configurable_profiler import UserConfigurableProfiler

Creamos una nueva suite de expectativas

In [81]:
expectation_suite_name = "nyc_data_profile"
suite = context.create_expectation_suite(
    expectation_suite_name
)

Primero debemos especificar como obtener este pequeño conjunto de datos. Note como ahora especificamos el nombre del origen de datos al que queremos aplicar el perfilado:

In [113]:
batch_request = ge.core.batch.BatchRequest(datasource_name="nyc_taxi", 
                                           data_connector_name="nyc_taxi_data_connector", 
                                           data_asset_name="yellow_tripdata_sample",
                                           limit=100)

Podemos obtener la muestra de datos:

In [114]:
context.get_batch_list(batch_request=batch_request)

Vemos que hay 2 lotes de datos disponibles:

In [110]:
context.get_batch_list(batch_request=batch_request)[0].batch_spec

{'path': '/home/santiagxf/repos/E72102/docs/develop/prep/code/great_expectations/../nyc_taxi/yellow_tripdata_sample_2019-01.csv'}

In [111]:
context.get_batch_list(batch_request=batch_request)[1].batch_spec

{'path': '/home/santiagxf/repos/E72102/docs/develop/prep/code/great_expectations/../nyc_taxi/yellow_tripdata_sample_2019-02.csv'}

Podemos obtener una muestra de los datos:

In [115]:
batch = context.get_batch_list(batch_request=batch_request)[0]
batch.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-15 03:36:12,2019-01-15 03:42:19,1,1.0,1,N,230,48,1,6.5,0.5,0.5,1.95,0.0,0.3,9.75,NaN
1,1,2019-01-25 18:20:32,2019-01-25 18:26:55,1,0.8,1,N,112,112,1,6.0,1.0,0.5,1.55,0.0,0.3,9.35,0.0
2,1,2019-01-05 06:47:31,2019-01-05 06:52:19,1,1.1,1,N,107,4,2,6.0,0.0,0.5,0.00,0.0,0.3,6.80,NaN
3,1,2019-01-09 15:08:02,2019-01-09 15:20:17,1,2.5,1,N,143,158,1,11.0,0.0,0.5,3.00,0.0,0.3,14.80,NaN
4,1,2019-01-25 18:49:51,2019-01-25 18:56:44,1,0.8,1,N,246,90,1,6.5,1.0,0.5,1.65,0.0,0.3,9.95,0.0


Ahora instanciamos nuestro generador del perfil de los datos:

In [122]:
profiler = UserConfigurableProfiler(profile_dataset=batch,
                                    ignored_columns=['vendor_id'])

Calculating Metrics:   0%|          | 0/3 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/3 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/3 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

Una vez que se generó el perfil de los datos, podemos constuir nuestra suite de expectativas:

In [123]:
suite = profiler.build_suite()

Profiling:   0%|          | 0/17 [00:00<?, ?it/s, Column=pickup_datetime]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

Creating an expectation suite with the following expectations:

Table-Level Expectations
expect_table_columns_to_match_ordered_list
expect_table_row_count_to_be_between

Expectations by Column
Column Name: congestion_surcharge | Column Data Type: FLOAT | Cardinality: ONE
expect_column_max_to_be_between
expect_column_mean_to_be_between
expect_column_median_to_be_between
expect_column_min_to_be_between
expect_column_proportion_of_unique_values_to_be_between
expect_column_quantile_values_to_be_between
expect_column_values_to_be_in_set
expect_column_values_to_be_in_type_list
expect_column_values_to_be_null


Column Name: dropoff_datetime | Column Data Type: STRING | Cardinality: VERY_MANY
expect_column_proportion_of_unique_values_to_be_between
expect_column_values_to_be_in_type_list
expect_column_values_to_not_be_null


Column Name: dropoff_location_id | Column Data Type: INT | Cardinality: MANY
expect_column_max_to_be_between
expect_column_mean_to_be_between
expect_column_median_to_be_bet

Guardamos nuestras expectativas

In [124]:
context.save_expectation_suite(suite, expectation_suite_name)

'/home/santiagxf/repos/E72102/docs/develop/prep/code/great_expectations/expectations/nyc_data_profile.json'

Inspeccionemos la suite:

In [147]:
suite.expectations[0].expectation_type

'expect_table_columns_to_match_ordered_list'

Vemos que la primera expectativa creada indica que las columnas en el conjunto de datos deben ser las siguientes, y en el orden esperado:

In [146]:
suite.expectations[0].kwargs["column_list"]

['vendor_id',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'rate_code_id',
 'store_and_fwd_flag',
 'pickup_location_id',
 'dropoff_location_id',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

> Importante: Cuando utilizamos `profiler`, todas las expectativas se aplican a todas las columnas. Esto puede resultar en una conjunto de validaciones demasiado rígidas que son dificiles que se repitan nuevamente en un conjunto de datos. Por ejemplo, una expectativa podría forzar a que la media de una columna sea exactamente 45.3. En algunos casos esto puede ser requerido, en otros no. En general, esta técnica se utiliza para generar una expectativa base sobre la que se puede ir iterando y refinando.

### Construyendo la documentación de expectativas

Una forma más amena de explorar estas expectativas es construyendo la documentación de las mismas. Esta documentación se genera automáticamente leyendo las expectativas de los datos que se generaron. Esto es una excelente forma de comunicar a los diferentes equipos de la organización lo que pueden esperar de estos datos:

In [148]:
context.build_data_docs()

{'local_site': 'file:///home/santiagxf/repos/E72102/docs/develop/prep/code/great_expectations/uncommitted/data_docs/local_site/index.html'}

Por ejemplo, la siguiente documentación se genera automáticamente:

<img src="../../_images/prep_ge_sample.png" />